In [ ]:
!pip install sentence-transformers

In [ ]:
from sentence_transformers import SentenceTransformer
from pyarrow.fs import S3FileSystem
import pyarrow.parquet as pq
import pyarrow as pa

# Initialize the model
model = SentenceTransformer("jinaai/jina-embeddings-v3", trust_remote_code=True)

# Initialize S3 filesystem
fs = S3FileSystem(access_key='B53zPFuC9ScfhJV9', secret_key='h7jAiZty4JqAGBSEN0M1LqgHyWc18otP', scheme='http', endpoint_override='10.0.0.21:32222')

In [ ]:
from delta.tables import DeltaTable

# Read master company table
master_companies_table = "s3a://salessmart-warehouse/table/master/company"
table = DeltaTable.forPath(spark, master_companies_table)
table_df = table.toDF()

# Prepare data for embedding
df = (
    table_df
        .select('corporate_number', 'name', 'business_content', 'meta_keywords', 'meta_descriptions', 'address')
        .filter(table_df['business_content'].isNotNull())
        .repartition(8000)
)

In [ ]:
# Process data in batches and generate embeddings
ids = []
contents = []
j = 0

for i, row in enumerate(df.toLocalIterator(True)):
    ids.append(row['corporate_number'])
    content = f"企業名：{row['name'] or ''}\n住所：{row['address'] or ''}\n企業概要：{row['business_content'] or ''}\nメタキーワード：{row['meta_keywords'] or ''}\nメタディスクリプション：{row['meta_descriptions'] or ''}"
    contents.append(content)
    
    if len(ids) >= 128:
        print(f'Processing file: {j}')
        document_embeddings = model.encode(contents, task='retrieval.passage')
        table = pa.table({
            'corporate_number': ids,
            'embedding': list(document_embeddings)
        })
        pq.write_table(table, f'salessmart-warehouse/test_company_embedding/{j}.parquet', filesystem=fs)
        j += 1
        ids = []
        contents = []